# 1. Initialiser Spark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("DatasetAnalysis") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

### Affichage des premières lignes

In [4]:
orders_data = spark.read.parquet('orders_data.parquet')
orders_data.toPandas().head()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin
0,2023-01-22 21:25:00,141234,iPhone,5.638009e+12,Vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000
1,2023-01-28 14:15:00,141235,Lightning Charging Cable,5.563320e+12,Alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750
2,2023-01-17 13:33:00,141236,Wired Headphones,2.113973e+12,Vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900
3,2023-01-05 20:33:00,141237,27in FHD Monitor,3.069157e+12,Sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965
4,2023-01-25 11:59:00,141238,Wired Headphones,9.692681e+12,Électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950


### Explorer la structure des données

In [5]:
# Voir le schéma et les types de données
orders_data.printSchema()

# Voir les dimensions
print(f"Nombre de lignes: {orders_data.count()}")
print(f"Nombre de colonnes: {len(orders_data.columns)}")

# Afficher les noms des colonnes
print("Colonnes:", orders_data.columns)

root
 |-- order_date: timestamp_ntz (nullable = true)
 |-- order_id: long (nullable = true)
 |-- product: string (nullable = true)
 |-- product_id: double (nullable = true)
 |-- category: string (nullable = true)
 |-- purchase_address: string (nullable = true)
 |-- quantity_ordered: long (nullable = true)
 |-- price_each: double (nullable = true)
 |-- cost_price: double (nullable = true)
 |-- turnover: double (nullable = true)
 |-- margin: double (nullable = true)

Nombre de lignes: 185950
Nombre de colonnes: 11
Colonnes: ['order_date', 'order_id', 'product', 'product_id', 'category', 'purchase_address', 'quantity_ordered', 'price_each', 'cost_price', 'turnover', 'margin']


### Statistiques descriptives

In [8]:
# Statistiques plus détaillées avec summary()
orders_data.summary().show()

+-------+-----------------+------------+--------------------+------------+--------------------+-------------------+------------------+------------------+------------------+------------------+
|summary|         order_id|     product|          product_id|    category|    purchase_address|   quantity_ordered|        price_each|        cost_price|          turnover|            margin|
+-------+-----------------+------------+--------------------+------------+--------------------+-------------------+------------------+------------------+------------------+------------------+
|  count|           185950|      185950|              185950|      185950|              185950|             185950|            185950|            185950|            185950|            185950|
|   mean|230417.5693788653|        NULL|5.509210670033572E12|        NULL|                NULL| 1.1243828986286637|184.39973476700638|  69.6685826527324|185.49091675145462|115.28942150794727|
| stddev|51512.73710999486|        NULL|

### Détecter les valeurs manquantes

In [10]:
# Compter les valeurs nulles par colonne (version corrigée)
from pyspark.sql.functions import col, sum, when, isnan, isnull

# Obtenir les types de colonnes
numeric_types = ["int", "bigint", "float", "double", "decimal"]
string_date_types = ["string", "timestamp", "date"]

# Fonction pour créer la condition appropriée selon le type
def null_condition(column_name, data_type):
    if any(t in data_type.lower() for t in numeric_types):
        # Pour les colonnes numériques, vérifier null ET nan
        return when(col(column_name).isNull() | isnan(col(column_name)), 1).otherwise(0)
    else:
        # Pour les autres types (string, date, etc.), seulement null
        return when(col(column_name).isNull(), 1).otherwise(0)

# Compter les valeurs nulles
null_counts = orders_data.select([
    sum(null_condition(c, dict(orders_data.dtypes)[c])).alias(c)
    for c in orders_data.columns
])
null_counts.show()

# Pourcentage de valeurs manquantes
total_rows = orders_data.count()
missing_percent = orders_data.select([
    (sum(null_condition(c, dict(orders_data.dtypes)[c])) / total_rows * 100).alias(c)
    for c in orders_data.columns
])
missing_percent.show()

+----------+--------+-------+----------+--------+----------------+----------------+----------+----------+--------+------+
|order_date|order_id|product|product_id|category|purchase_address|quantity_ordered|price_each|cost_price|turnover|margin|
+----------+--------+-------+----------+--------+----------------+----------------+----------+----------+--------+------+
|         0|       0|      0|         0|       0|               0|               0|         0|         0|       0|     0|
+----------+--------+-------+----------+--------+----------------+----------------+----------+----------+--------+------+

+----------+--------+-------+----------+--------+----------------+----------------+----------+----------+--------+------+
|order_date|order_id|product|product_id|category|purchase_address|quantity_ordered|price_each|cost_price|turnover|margin|
+----------+--------+-------+----------+--------+----------------+----------------+----------+----------+--------+------+
|       0.0|     0.0|  

### Détecter les doublons

In [14]:
# Nombre total de doublons
print(f"Lignes uniques: {orders_data.distinct().count()}")
print(f"Doublons: {orders_data.count() - orders_data.distinct().count()}")

Lignes uniques: 185950
Doublons: 0
